<ul class="breadcrumb">
  <li><a href="1.1.Univaraite_Linear_Mixed_Effects_Model.ipynb">Univariate Basics</a></li>
  <li><a href="1.2.Univariate_Linear_Additive_Genetic_Model.ipynb">Univariate Additive Genetic Model</a></li> 
  <li><a href="1.3.Univariate_Linear_Mixed_Effects_Model_with_Genomic_Data.ipynb">Univariate Genomic Data</a></li>
</ul>

<div class="span5 alert alert-success">
 <font size="5" face="Georgia">Univaraite Linear Mixed Effects Model</font> 
</div>

In [1]:
using DataFrames
using JWAS: ST,Datasets

<button type="button" class="btn btn-lg btn-primary">Data</button> 

In [2]:
phenofile = Datasets.dataset("testMME","data.txt")
data      = readtable(phenofile,separator = ',',header=true)

,sow,site,yr,age,geneticCode,parity,nwn,SYS
1,100-113,113,2005,18,PIC 1,1,8,113_2005_WNTR
2,100-113,113,2006,18,PIC 1,2,12,113_2006_SPNG
3,100-5,5,2008,15,PIC 2,1,10,5_2008_ATMN
4,1000-5,5,2009,17,PIC 2,1,10,5_2009_SPNG
5,10000-13,13,2004,16,Commercial,1,9,13_2004_WNTR
6,10000-13,13,2004,18,Commercial,2,10,13_2004_SMMR
7,10000-13,13,2004,20,Commercial,3,11,13_2004_ATMN
8,10000-13,13,2005,18,Commercial,4,11,13_2005_SPNG
9,10000-13,13,2005,25,Commercial,5,7,13_2005_ATMN
10,10000-6,6,2012,17,PIC C27,1,12,6_2012_ATMN


<button type="button" class="btn btn-lg btn-primary">Build Model</button> 

In [3]:
model  = "nwn = intercept + parity + site + yr + geneticCode + age + sow + SYS"
varRes = 2.97      # starting value for residual variance
mme    = build_model(model,varRes)

set_covariate(mme,"age")

ν      = 4.0       # degrees of freedom for prior of variance components
v1     = 0.26      # starting value for sow variance
set_random(mme,"sow",v1,ν)
v2     = 0.28      # starting value for SYS variance
set_random(mme,"SYS",v1,ν);

<button type="button" class="btn btn-lg btn-primary">Run Model</button> 

In [4]:
outputMCMCsamples(mme,"parity","age");
out=runMCMC(mme,data,chain_length=50000,printout_frequency=5000);

at sample: 5000 with meanVare: 2.9739756374170887
at sample: 10000 with meanVare: 2.9199789695308236
at sample: 15000 with meanVare: 2.8835335066849686
at sample: 20000 with meanVare: 2.8899968667954763
at sample: 25000 with meanVare: 2.8594303629971565
at sample: 30000 with meanVare: 2.8822272461623193
at sample: 35000 with meanVare: 2.9213102723033684
at sample: 40000 with meanVare: 2.972056840551057
at sample: 45000 with meanVare: 2.982091421665865
at sample: 50000 with meanVare: 2.9700848005736766


<button type="button" class="btn btn-lg btn-primary">Check Results</button> 

In [5]:
keys(out)

Base.KeyIterator for a Dict{Any,Any} with 6 entries. Keys:
  "Posterior Mean of Location Parameters"
  "MCMC samples for residual variance"
  "MCMC samples for: parity"
  "MCMC samples for: variance of SYS"
  "MCMC samples for: age"
  "MCMC samples for: variance of sow"

In [ ]:
using Gadfly
sample4age = out["MCMC samples for: age"]
plot(x=sample4age[30000:end], Geom.histogram)